# K-Means with Dynamic Time Warping

using LB Keogh lower bound of dynamic time warping
http://alexminnaar.com/time-series-classification-and-clustering-with-python.html

In [ ]:
!pip install cython
!pip install tslearn

In [53]:
from sqlalchemy import create_engine
import os

def create_connection():
    host = 'localhost'
    port = '3333'
    return create_engine("postgresql://infocup@{}:{}/infocup".format(host,port))

In [517]:
con = create_connection()

active_stations = pd.read_sql_query("""SELECT station_id,time_stamp, price 
FROM prices_sampled 
WHERE price > 0 AND station_id <= 1000 AND date_part('hour', time_stamp)::int %% 8 = 0
AND time_stamp >= '2017-06-01'""", con)
active_stations['time_stamp'] = active_stations['time_stamp'].astype(np.int64) // 10**9
active_stations.tail(10)

,station_id,time_stamp,price
323874,1000,1505750400,1309
323875,1000,1505779200,1409
323876,1000,1505808000,1389
323877,1000,1505836800,1309
323878,1000,1505865600,1419
323879,1000,1505894400,1389
323880,1000,1505923200,1309
323881,1000,1505952000,1419
323882,1000,1505980800,1379
323883,1000,1506009600,1329


In [586]:
active_stations = active_stations[np.isfinite(active_stations['price'])]
active_stations.dropna(inplace=True)
active_stations.loc[np.isnan(active_stations['price'])]
prices_pivoted = active_stations.pivot_table(index='time_stamp', columns='station_id', values='price',fill_value=0)


In [589]:
from tslearn.clustering import TimeSeriesKMeans
km = TimeSeriesKMeans(n_clusters=10, metric="dtw")
km.fit(prices_pivoted[0:15].as_matrix())

56015.067 --> 31732.587 --> 31732.587 --> 


TimeSeriesKMeans(dtw_inertia=False, max_iter=50, max_iter_barycenter=100,
         metric='dtw', metric_params=None, n_clusters=10, n_init=1,
         random_state=None, tol=1e-06, verbose=True)

In [ ]:
y_pred = km.predict(prices_pivoted[16:22].as_matrix())

In [601]:
y_pred

array([3, 0, 6, 3, 8, 6])